In [76]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from keras import models, Input
from keras import optimizers as opt
from keras import backend as K
from keras.layers import Dense
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.utils import to_categorical

In [77]:
from dataset import load_dataset, load_labels, convert_to_epochs, format_labels
from features import time_series_features, fractal_features, entropy_features, hjorth_features, freq_band_features

# Variables

In [78]:
n_classes = 2
n_channels = 32
sfreq = 128

In [79]:
test_type = "Arithmetic"

# Load Dataset

In [80]:
dataset_ = load_dataset(raw=False, test_type=test_type)
dataset = convert_to_epochs(dataset_, n_channels, sfreq)

In [81]:
label_ = load_labels()
label = format_labels(label_, test_type=test_type, epochs=dataset.shape[1])

# Compute Features

In [82]:
features = time_series_features(dataset, n_channels)
# freq_bands = np.array([1, 4, 8, 13, 31, 50])
# features = freq_band_features(dataset, channels, sfreq, freq_bands)
# features = fractal_features(dataset, channels)
# features = hjorth_features(dataset, channels, sfreq)
# features = entropy_features(dataset, channels, sfreq)
data = features

# k-NN Classifier

In [83]:
K.clear_session()
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
neigh_clf = KNeighborsClassifier(n_neighbors=5)
neigh_clf.fit(x_train, y_train)

KNeighborsClassifier()

In [84]:
y_pred = neigh_clf.predict(x_val)
y_true = y_val

In [85]:
scores_neigh = neigh_clf.score(x_test, y_test)
precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
print("--------------")
print("VALIDATION SET")
print("--------------")
print('accuracy is:', scores_neigh)
print('precision is:', precision_neigh)
print('recall is:', recall_neigh)
print('f1_score is:', f1_score_neigh)
metrics.confusion_matrix(y_true, y_pred)


--------------
VALIDATION SET
--------------
accuracy is: 0.6783333333333333
precision is: 0.47828234254373425
recall is: 0.45666666666666667
f1_score is: 0.3704125


array([[ 37, 290],
       [ 36, 237]])

In [86]:
y_pred = neigh_clf.predict(x_test)
y_true = y_test

In [87]:
scores_neigh = neigh_clf.score(x_test, y_test)
precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
print("--------")
print("TEST SET")
print("--------")
print('accuracy is:', scores_neigh)
print('precision is:', precision_neigh)
print('recall is:', recall_neigh)
print('f1_score is:', f1_score_neigh)
metrics.confusion_matrix(y_true, y_pred)


--------
TEST SET
--------
accuracy is: 0.6783333333333333
precision is: 0.6809581320450886
recall is: 0.6783333333333333
f1_score is: 0.6779265914646968


array([[197, 114],
       [ 79, 210]])

# SVM Classifier

In [88]:
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)
svm_clf = SVC(C=100, kernel='rbf')
svm_clf.fit(x_train, y_train)

SVC(C=100)

In [89]:
y_pred = svm_clf.predict(x_val)
y_true = y_val

In [90]:
scores_svm = svm_clf.score(x_val, y_val)
precision_svm = metrics.precision_score(y_true, y_pred, average='macro')
recall_svm = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_svm = metrics.f1_score(y_true, y_pred, average='weighted')
print("--------------")
print("VALIDATION SET")
print("--------------")
print('accuracy is:', scores_svm)
print('precision is:', precision_svm)
print('recall is:', recall_svm)
print('f1_score is:', f1_score_svm)
metrics.confusion_matrix(y_true, y_pred)


--------------
VALIDATION SET
--------------
accuracy is: 0.8316666666666667
precision is: 0.8310335497835498
recall is: 0.8316666666666667
f1_score is: 0.8313985860781785


array([[281,  46],
       [ 55, 218]])

In [91]:
y_pred = svm_clf.predict(x_test)
y_true = y_test

In [92]:
scores_svm = svm_clf.score(x_test, y_test)
precision_svm = metrics.precision_score(y_true, y_pred, average='macro')
recall_svm = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_svm = metrics.f1_score(y_true, y_pred, average='weighted')
print("--------")
print("TEST SET")
print("--------")
print('accuracy is:', scores_svm)
print('precision is:', precision_svm)
print('recall is:', recall_svm)
print('f1_score is:', f1_score_svm)
metrics.confusion_matrix(y_true, y_pred)


--------
TEST SET
--------
accuracy is: 0.8533333333333334
precision is: 0.8542657342657343
recall is: 0.8533333333333334
f1_score is: 0.8531272581292655


array([[274,  37],
       [ 51, 238]])

# Multilayer Perceptron

In [93]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, n_classes)
x_train, x_test, y_train, y_test = train_test_split(data, y_v, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

In [94]:
def model_builder(hp):
    model = models.Sequential()
    model.add(Input(shape=(x_train.shape[1],)))

    for i in range(hp.Int('layers', 2, 6)):
      model.add(Dense(units=hp.Int('units_' + str(i), 32, 1024, step=32),
                                    activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

    model.add(Dense(n_classes, activation='softmax', name='out'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=opt.adam_v2.Adam(learning_rate=hp_learning_rate),
                loss = "binary_crossentropy",
                metrics=['accuracy'])
    return model

In [95]:
tuner = RandomSearch(
    model_builder,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 2,
    overwrite=True
)

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-11-10 10:58:09.157824: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-10 10:58:09.158304: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [96]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [97]:
tuner.search(x_train, y_train, epochs = 50, validation_data= [x_val, y_val])


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
4                 |?                 |layers
192               |?                 |units_0
sigmoid           |?                 |act_0
64                |?                 |units_1
relu              |?                 |act_1
0.0001            |?                 |learning_rate

Epoch 1/50


2022-11-10 10:58:09.482463: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-10 10:58:09.646593: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


57/57 [==============================] - ETA: 0s - loss: 0.6879 - accuracy: 0.5650

2022-11-10 10:58:10.637048: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


57/57 [==============================] - 1s 17ms/step - loss: 0.6879 - accuracy: 0.5650 - val_loss: 0.6789 - val_accuracy: 0.5817
Epoch 2/50
57/57 [==============================] - 1s 15ms/step - loss: 0.6703 - accuracy: 0.6233 - val_loss: 0.6633 - val_accuracy: 0.6633
Epoch 3/50
57/57 [==============================] - 1s 16ms/step - loss: 0.6569 - accuracy: 0.6606 - val_loss: 0.6519 - val_accuracy: 0.6700
Epoch 4/50
57/57 [==============================] - 1s 13ms/step - loss: 0.6445 - accuracy: 0.6767 - val_loss: 0.6439 - val_accuracy: 0.6483
Epoch 5/50
57/57 [==============================] - 1s 12ms/step - loss: 0.6334 - accuracy: 0.6922 - val_loss: 0.6374 - val_accuracy: 0.6183
Epoch 6/50
57/57 [==============================] - 1s 12ms/step - loss: 0.6240 - accuracy: 0.6817 - val_loss: 0.6243 - val_accuracy: 0.6917
Epoch 7/50
57/57 [==============================] - 1s 11ms/step - loss: 0.6091 - accuracy: 0.7128 - val_loss: 0.6140 - val_accuracy: 0.6817
Epoch 8/50
57/57 [======

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

2022-11-09 08:48:06.981466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [ ]:
scores_dnn = model.evaluate(x_test, y_test, verbose=0)
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print("--------")
print("TEST SET")
print("--------")
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)

NameError: name 'model' is not defined